In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd drive/My Drive/prepro

/content/drive/My Drive/prepro


In [ ]:
!ls

calendar.csv  new_data.csv     prepro.ipynb		   sell_prices.csv
dataset.csv   prepro_data.csv  sales_train_evaluation.csv


In [ ]:
import torch
import torch.optim as optim
import numpy as np
import os
import pandas as pd

In [ ]:
calendar = pd.read_csv('calendar.csv')

In [ ]:
sell_prices = pd.read_csv('sell_prices.csv')

In [ ]:
sales_train_evaluation = pd.read_csv('sales_train_evaluation.csv')

In [ ]:
sales_train_evaluation_melt = pd.melt(sales_train_evaluation, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='day', value_name='demand')

In [ ]:
sales_CA_1 = sales_train_evaluation_melt
print(sales_CA_1)

                                    id        item_id    dept_id   cat_id  \
0        HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1        HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2        HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3        HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4        HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                                ...            ...        ...      ...   
5918104    FOODS_3_823_CA_1_evaluation    FOODS_3_823    FOODS_3    FOODS   
5918105    FOODS_3_824_CA_1_evaluation    FOODS_3_824    FOODS_3    FOODS   
5918106    FOODS_3_825_CA_1_evaluation    FOODS_3_825    FOODS_3    FOODS   
5918107    FOODS_3_826_CA_1_evaluation    FOODS_3_826    FOODS_3    FOODS   
5918108    FOODS_3_827_CA_1_evaluation    FOODS_3_827    FOODS_3    FOODS   

        store_id state_id         day  demand  
0           CA_1       CA  

In [ ]:
new_CA_1 = pd.merge(sales_CA_1, calendar, left_on="day", right_on="date", how="left")
print(new_CA_1)

                                    id        item_id    dept_id   cat_id  \
0        HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1        HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2        HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3        HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4        HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                                ...            ...        ...      ...   
5918104    FOODS_3_823_CA_1_evaluation    FOODS_3_823    FOODS_3    FOODS   
5918105    FOODS_3_824_CA_1_evaluation    FOODS_3_824    FOODS_3    FOODS   
5918106    FOODS_3_825_CA_1_evaluation    FOODS_3_825    FOODS_3    FOODS   
5918107    FOODS_3_826_CA_1_evaluation    FOODS_3_826    FOODS_3    FOODS   
5918108    FOODS_3_827_CA_1_evaluation    FOODS_3_827    FOODS_3    FOODS   

        store_id state_id         day  demand        date  wm_yr_wk  ...  \

In [ ]:
new_CA_1 = pd.merge(new_CA_1, sell_prices, left_on=["store_id", "item_id", "wm_yr_wk"],right_on=["store_id", "item_id", "wm_yr_wk"], how="left")

In [ ]:
print(new_CA_1)

                                    id        item_id    dept_id   cat_id  \
0        HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1        HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2        HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3        HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4        HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                                ...            ...        ...      ...   
5918104    FOODS_3_823_CA_1_evaluation    FOODS_3_823    FOODS_3    FOODS   
5918105    FOODS_3_824_CA_1_evaluation    FOODS_3_824    FOODS_3    FOODS   
5918106    FOODS_3_825_CA_1_evaluation    FOODS_3_825    FOODS_3    FOODS   
5918107    FOODS_3_826_CA_1_evaluation    FOODS_3_826    FOODS_3    FOODS   
5918108    FOODS_3_827_CA_1_evaluation    FOODS_3_827    FOODS_3    FOODS   

        store_id state_id         day  demand        date  wm_yr_wk  ...  \

In [ ]:
new_CA_1.to_csv("dataset.csv")

In [ ]:
import datetime as dt

In [ ]:
def date_features(df):
    
    df["date"] = pd.to_datetime(df["date"])
    df["day"] = df.date.dt.day
    df["month"] = df.date.dt.month
    df["week_day"] = df.date.dt.weekday

    df.drop(columns="date", inplace=True)

    return df

def sales_features(df):

    df.sell_price.fillna(0, inplace=True)

    return df

def demand_features(df):

    df["lag_t28"] = df["demand"].transform(lambda x: x.shift(28))
    df["rolling_mean_t7"] = df["demand"].transform(lambda x:x.shift(28).rolling(7).mean())
    df['rolling_mean_t30'] = df['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
    df['rolling_mean_t60'] = df['demand'].transform(lambda x: x.shift(28).rolling(60).mean())
    df['rolling_mean_t90'] = df['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
    df['rolling_mean_t180'] = df['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
    df['rolling_std_t7'] = df['demand'].transform(lambda x: x.shift(28).rolling(7).std())
    df['rolling_std_t30'] = df['demand'].transform(lambda x: x.shift(28).rolling(30).std())

    df.fillna(0, inplace=True)

    return df

In [ ]:
date_features(new_CA_1)

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,wm_yr_wk,weekday,...,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,week_day
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,d_1,NaN,NaN,NaN,NaN,0,0,0,NaN,5
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,d_1,NaN,NaN,NaN,NaN,0,0,0,NaN,5
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,d_1,NaN,NaN,NaN,NaN,0,0,0,NaN,5
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,d_1,NaN,NaN,NaN,NaN,0,0,0,NaN,5
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,d_1,NaN,NaN,NaN,NaN,0,0,0,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5918104,FOODS_3_823_CA_1_evaluation,FOODS_3_823,FOODS_3,FOODS,CA_1,CA,22,2,11617,Sunday,...,d_1941,NaN,NaN,NaN,NaN,0,0,0,2.98,6
5918105,FOODS_3_824_CA_1_evaluation,FOODS_3_824,FOODS_3,FOODS,CA_1,CA,22,0,11617,Sunday,...,d_1941,NaN,NaN,NaN,NaN,0,0,0,2.48,6
5918106,FOODS_3_825_CA_1_evaluation,FOODS_3_825,FOODS_3,FOODS,CA_1,CA,22,1,11617,Sunday,...,d_1941,NaN,NaN,NaN,NaN,0,0,0,3.98,6
5918107,FOODS_3_826_CA_1_evaluation,FOODS_3_826,FOODS_3,FOODS,CA_1,CA,22,1,11617,Sunday,...,d_1941,NaN,NaN,NaN,NaN,0,0,0,1.28,6


In [ ]:
print(new_CA_1)

                                    id        item_id    dept_id   cat_id  \
0        HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1        HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2        HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3        HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4        HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                                ...            ...        ...      ...   
5918104    FOODS_3_823_CA_1_evaluation    FOODS_3_823    FOODS_3    FOODS   
5918105    FOODS_3_824_CA_1_evaluation    FOODS_3_824    FOODS_3    FOODS   
5918106    FOODS_3_825_CA_1_evaluation    FOODS_3_825    FOODS_3    FOODS   
5918107    FOODS_3_826_CA_1_evaluation    FOODS_3_826    FOODS_3    FOODS   
5918108    FOODS_3_827_CA_1_evaluation    FOODS_3_827    FOODS_3    FOODS   

        store_id state_id  day  demand  wm_yr_wk   weekday  ...       d  \


In [ ]:
print(new_CA_1['month'])

0          1
1          1
2          1
3          1
4          1
          ..
5918104    5
5918105    5
5918106    5
5918107    5
5918108    5
Name: month, Length: 5918109, dtype: int64


In [ ]:
sales_features(new_CA_1)

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,wm_yr_wk,weekday,...,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,week_day
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,d_1,NaN,NaN,NaN,NaN,0,0,0,0.00,5
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,d_1,NaN,NaN,NaN,NaN,0,0,0,0.00,5
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,d_1,NaN,NaN,NaN,NaN,0,0,0,0.00,5
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,d_1,NaN,NaN,NaN,NaN,0,0,0,0.00,5
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,d_1,NaN,NaN,NaN,NaN,0,0,0,0.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5918104,FOODS_3_823_CA_1_evaluation,FOODS_3_823,FOODS_3,FOODS,CA_1,CA,22,2,11617,Sunday,...,d_1941,NaN,NaN,NaN,NaN,0,0,0,2.98,6
5918105,FOODS_3_824_CA_1_evaluation,FOODS_3_824,FOODS_3,FOODS,CA_1,CA,22,0,11617,Sunday,...,d_1941,NaN,NaN,NaN,NaN,0,0,0,2.48,6
5918106,FOODS_3_825_CA_1_evaluation,FOODS_3_825,FOODS_3,FOODS,CA_1,CA,22,1,11617,Sunday,...,d_1941,NaN,NaN,NaN,NaN,0,0,0,3.98,6
5918107,FOODS_3_826_CA_1_evaluation,FOODS_3_826,FOODS_3,FOODS,CA_1,CA,22,1,11617,Sunday,...,d_1941,NaN,NaN,NaN,NaN,0,0,0,1.28,6


In [ ]:
demand_features(new_CA_1)

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,wm_yr_wk,weekday,...,sell_price,week_day,lag_t28,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_std_t7,rolling_std_t30
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,0.00,5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,0.00,5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,0.00,5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,0.00,5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,Saturday,...,0.00,5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5918104,FOODS_3_823_CA_1_evaluation,FOODS_3_823,FOODS_3,FOODS,CA_1,CA,22,2,11617,Sunday,...,2.98,6,4.0,3.714286,4.166667,4.883333,4.655556,4.111111,2.690371,4.579439
5918105,FOODS_3_824_CA_1_evaluation,FOODS_3_824,FOODS_3,FOODS,CA_1,CA,22,0,11617,Sunday,...,2.48,6,0.0,2.571429,4.133333,4.883333,4.566667,4.105556,2.225395,4.606841
5918106,FOODS_3_825_CA_1_evaluation,FOODS_3_825,FOODS_3,FOODS,CA_1,CA,22,1,11617,Sunday,...,3.98,6,2.0,2.857143,4.200000,4.916667,4.588889,4.116667,1.951800,4.559189
5918107,FOODS_3_826_CA_1_evaluation,FOODS_3_826,FOODS_3,FOODS,CA_1,CA,22,1,11617,Sunday,...,1.28,6,0.0,2.571429,3.900000,4.900000,4.555556,4.111111,2.225395,4.528454


In [ ]:
new_CA_1.to_csv("new_data.csv")

In [ ]:
df = pd.read_csv("new_data.csv")

In [ ]:
df.head(10)

,Unnamed: 0,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,wm_yr_wk,...,sell_price,week_day,lag_t28,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_std_t7,rolling_std_t30
0,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,...,0.00,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,...,0.00,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,...,0.00,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,...,0.00,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,...,0.00,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,...,0.00,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,...,0.00,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,29,12,11101,...,0.46,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,29,2,11101,...,1.56,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,29,0,11101,...,3.17,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918109 entries, 0 to 5918108
Data columns (total 32 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   id                 object 
 2   item_id            object 
 3   dept_id            object 
 4   cat_id             object 
 5   store_id           object 
 6   state_id           object 
 7   day                int64  
 8   demand             int64  
 9   wm_yr_wk           int64  
 10  weekday            object 
 11  wday               int64  
 12  month              int64  
 13  year               int64  
 14  d                  object 
 15  event_name_1       object 
 16  event_type_1       object 
 17  event_name_2       object 
 18  event_type_2       object 
 19  snap_CA            int64  
 20  snap_TX            int64  
 21  snap_WI            int64  
 22  sell_price         float64
 23  week_day           int64  
 24  lag_t28            float64
 25  rolling_mean_t7   

In [ ]:
print(df['wday'])

0          1
1          1
2          1
3          1
4          1
          ..
5918104    2
5918105    2
5918106    2
5918107    2
5918108    2
Name: wday, Length: 5918109, dtype: int64


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
from tqdm import tqdm
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
cat_id_cols = ["item_id", "dept_id", "store_id", "cat_id", "state_id"]
cat_cols = cat_id_cols + ["wday", "month", "year", "event_name_1", 
                          "event_type_1", "event_name_2", "event_type_2"]

# In loop to minimize memory use
for i, v in tqdm(enumerate(cat_id_cols)):
    df[v] = OrdinalEncoder(dtype="int").fit_transform(df[[v]])

df = reduce_mem_usage(df)
df.head()

5it [00:02,  1.85it/s]


Mem. usage decreased to 547.46 Mb (26.0% reduction)


,Unnamed: 0,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,wm_yr_wk,...,sell_price,week_day,lag_t28,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_std_t7,rolling_std_t30
0,0,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,29,0,11101,...,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,HOBBIES_1_002_CA_1_evaluation,1438,3,1,0,0,29,0,11101,...,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,HOBBIES_1_003_CA_1_evaluation,1439,3,1,0,0,29,0,11101,...,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,HOBBIES_1_004_CA_1_evaluation,1440,3,1,0,0,29,0,11101,...,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,HOBBIES_1_005_CA_1_evaluation,1441,3,1,0,0,29,0,11101,...,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
num_cols = ["sell_price", "lag_t28", "rolling_mean_t7", "rolling_mean_t30", "rolling_mean_t60", "rolling_mean_t90", "rolling_mean_t180", "rolling_std_t7", "rolling_std_t30"]
bool_cols = ["snap_CA", "snap_TX", "snap_WI"]
dense_cols = num_cols + bool_cols

In [ ]:
for i, v in tqdm(enumerate(num_cols)):
    df[v] = df[v].fillna(df[v].median())
    
df.head()

9it [00:02,  4.33it/s]


,Unnamed: 0,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,wm_yr_wk,...,sell_price,week_day,lag_t28,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_std_t7,rolling_std_t30
0,0,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,29,0,11101,...,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,HOBBIES_1_002_CA_1_evaluation,1438,3,1,0,0,29,0,11101,...,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,HOBBIES_1_003_CA_1_evaluation,1439,3,1,0,0,29,0,11101,...,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,HOBBIES_1_004_CA_1_evaluation,1440,3,1,0,0,29,0,11101,...,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,HOBBIES_1_005_CA_1_evaluation,1441,3,1,0,0,29,0,11101,...,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df['dn'] = df['d'].str.extract(r'(\d+)')

In [ ]:
df.to_csv("prepro_data.csv")

In [ ]:
df['dn'] = pd.to_numeric(df['dn'])

In [ ]:
print(df['dn'])

0             1
1             1
2             1
3             1
4             1
           ... 
5918104    1941
5918105    1941
5918106    1941
5918107    1941
5918108    1941
Name: dn, Length: 5918109, dtype: int64


In [ ]:
test = df[df.dn >= 1914]
test = test.assign(id=test.id + "_" + np.where(test.dn <= 1941, "validation", "evaluation"),
                   F="F" + (test.dn - 1913 - 28 * (test.dn > 1941)).astype("str"))
test.head()

,Unnamed: 0,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,wm_yr_wk,...,lag_t28,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_std_t7,rolling_std_t30,dn,F
5832737,5832737,HOBBIES_1_001_CA_1_evaluation_validation,1437,3,1,0,0,25,0,11613,...,1.0,2.000000,4.000000,4.617188,4.179688,3.839844,2.449219,5.734375,1914,F1
5832738,5832738,HOBBIES_1_002_CA_1_evaluation_validation,1438,3,1,0,0,25,0,11613,...,2.0,1.857422,3.933594,4.250000,4.187500,3.810547,2.410156,5.746094,1914,F1
5832739,5832739,HOBBIES_1_003_CA_1_evaluation_validation,1439,3,1,0,0,25,0,11613,...,0.0,0.856934,3.632812,4.214844,4.179688,3.804688,0.899902,5.707031,1914,F1
5832740,5832740,HOBBIES_1_004_CA_1_evaluation_validation,1440,3,1,0,0,25,0,11613,...,11.0,2.285156,3.933594,4.382812,4.300781,3.867188,3.945312,5.855469,1914,F1
5832741,5832741,HOBBIES_1_005_CA_1_evaluation_validation,1441,3,1,0,0,25,1,11613,...,2.0,2.285156,4.000000,4.183594,4.320312,3.873047,3.945312,5.820312,1914,F1


In [ ]:
def make_X(df):
    X = {"dense1": df[dense_cols].to_numpy()}
    for i, v in enumerate(cat_cols):
        X[v] = df[[v]].to_numpy()
    return X

# Submission data
X_test = make_X(test)

# One month of validation data
flag = (df.dn < 1914) & (df.dn >= 1914 - 28)
valid = (make_X(df[flag]),
         df["demand"][flag])

# Rest is used for training
flag = df.dn < 1914 - 28
X_train = make_X(df[flag])
y_train = df["demand"][flag]                            

In [ ]:
df = pd.read_csv("prepro_data.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
pip install keras-tcn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.9 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, concatenate, Flatten
from tensorflow.keras.models import Model

from tcn import TCN

In [ ]:

# Dense input
dense_input = Input(shape=(len(dense_cols), ), name='dense1')

# Embedding input
wday_input = Input(shape=(1,), name='wday')
month_input = Input(shape=(1,), name='month')
year_input = Input(shape=(1,), name='year')
event_name_1_input = Input(shape=(1,), name='event_name_1')
event_type_1_input = Input(shape=(1,), name='event_type_1')
event_name_2_input = Input(shape=(1,), name='event_name_2')
event_type_2_input = Input(shape=(1,), name='event_type_2')
item_id_input = Input(shape=(1,), name='item_id')
dept_id_input = Input(shape=(1,), name='dept_id')
store_id_input = Input(shape=(1,), name='store_id')
cat_id_input = Input(shape=(1,), name='cat_id')
state_id_input = Input(shape=(1,), name='state_id')

wday_emb = Flatten()(Embedding(7, 1)(wday_input))
month_emb = Flatten()(Embedding(12, 1)(month_input))
year_emb = Flatten()(Embedding(6, 1)(year_input))
event_name_1_emb = Flatten()(Embedding(31, 1)(event_name_1_input))
event_type_1_emb = Flatten()(Embedding(5, 1)(event_type_1_input))
event_name_2_emb = Flatten()(Embedding(5, 1)(event_name_2_input))
event_type_2_emb = Flatten()(Embedding(5, 1)(event_type_2_input))

item_id_emb = Flatten()(Embedding(3049, 3)(item_id_input))
dept_id_emb = Flatten()(Embedding(7, 1)(dept_id_input))
store_id_emb = Flatten()(Embedding(10, 1)(store_id_input))
cat_id_emb = Flatten()(Embedding(3, 1)(cat_id_input))
state_id_emb = Flatten()(Embedding(3, 1)(state_id_input))

# Combine dense and embedding parts and add dense layers. Exit on linear scale.
x = concatenate([dense_input, wday_emb, month_emb, year_emb, 
                event_name_1_emb, event_type_1_emb, 
                event_name_2_emb, event_type_2_emb, 
                item_id_emb, dept_id_emb, store_id_emb,
                cat_id_emb, state_id_emb])

inputs = {"dense1": dense_input, "wday": wday_input, "month": month_input, "year": year_input, 
          "event_name_1": event_name_1_input, "event_type_1": event_type_1_input,
          "event_name_2": event_name_2_input, "event_type_2": event_type_2_input,
          "item_id": item_id_input, "dept_id": dept_id_input, "store_id": store_id_input, 
          "cat_id": cat_id_input, "state_id": state_id_input}

In [ ]:
print(x)

KerasTensor(type_spec=TensorSpec(shape=(None, 26), dtype=tf.float32, name=None), name='concatenate_1/concat:0', description="created by layer 'concatenate_1'")


In [ ]:
train = df[df.dn < 1914]

In [ ]:
print(train)

         Unnamed: 0                             id  item_id  dept_id  cat_id  \
0                 0  HOBBIES_1_001_CA_1_evaluation     1437        3       1   
1                 1  HOBBIES_1_002_CA_1_evaluation     1438        3       1   
2                 2  HOBBIES_1_003_CA_1_evaluation     1439        3       1   
3                 3  HOBBIES_1_004_CA_1_evaluation     1440        3       1   
4                 4  HOBBIES_1_005_CA_1_evaluation     1441        3       1   
...             ...                            ...      ...      ...     ...   
5832732     5832732    FOODS_3_823_CA_1_evaluation     1432        2       0   
5832733     5832733    FOODS_3_824_CA_1_evaluation     1433        2       0   
5832734     5832734    FOODS_3_825_CA_1_evaluation     1434        2       0   
5832735     5832735    FOODS_3_826_CA_1_evaluation     1435        2       0   
5832736     5832736    FOODS_3_827_CA_1_evaluation     1436        2       0   

         store_id  state_id  day  deman

In [ ]:
test = df[df.dn >= 1914]
print(test)

         Unnamed: 0  Unnamed: 0.1                             id  item_id  \
5832737     5832737       5832737  HOBBIES_1_001_CA_1_evaluation     1437   
5832738     5832738       5832738  HOBBIES_1_002_CA_1_evaluation     1438   
5832739     5832739       5832739  HOBBIES_1_003_CA_1_evaluation     1439   
5832740     5832740       5832740  HOBBIES_1_004_CA_1_evaluation     1440   
5832741     5832741       5832741  HOBBIES_1_005_CA_1_evaluation     1441   
...             ...           ...                            ...      ...   
5918104     5918104       5918104    FOODS_3_823_CA_1_evaluation     1432   
5918105     5918105       5918105    FOODS_3_824_CA_1_evaluation     1433   
5918106     5918106       5918106    FOODS_3_825_CA_1_evaluation     1434   
5918107     5918107       5918107    FOODS_3_826_CA_1_evaluation     1435   
5918108     5918108       5918108    FOODS_3_827_CA_1_evaluation     1436   

         dept_id  cat_id  store_id  state_id  day  demand  ...  week_day  \

In [ ]:
X_train = train.drop(columns=["dn", "d", "wm_yr_wk", "Unnamed: 0", "Unnamed: 0.1",  "id", "day", "weekday", "demand", "event_name_1", "event_type_1", "event_name_2", "event_type_2"], axis=1)
y_train = train[["demand"]]

In [ ]:
print(X_train)

         item_id  dept_id  cat_id  store_id  state_id  wday  month  year  \
0           1437        3       1         0         0     1      1  2011   
1           1438        3       1         0         0     1      1  2011   
2           1439        3       1         0         0     1      1  2011   
3           1440        3       1         0         0     1      1  2011   
4           1441        3       1         0         0     1      1  2011   
...          ...      ...     ...       ...       ...   ...    ...   ...   
5832732     1432        2       0         0         0     2      4  2016   
5832733     1433        2       0         0         0     2      4  2016   
5832734     1434        2       0         0         0     2      4  2016   
5832735     1435        2       0         0         0     2      4  2016   
5832736     1436        2       0         0         0     2      4  2016   

         snap_CA  snap_TX  ...  sell_price  week_day  lag_t28  \
0              0      

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5832737 entries, 0 to 5832736
Data columns (total 21 columns):
 #   Column             Dtype  
---  ------             -----  
 0   item_id            int64  
 1   dept_id            int64  
 2   cat_id             int64  
 3   store_id           int64  
 4   state_id           int64  
 5   wday               int64  
 6   month              int64  
 7   year               int64  
 8   snap_CA            int64  
 9   snap_TX            int64  
 10  snap_WI            int64  
 11  sell_price         float64
 12  week_day           int64  
 13  lag_t28            float64
 14  rolling_mean_t7    float64
 15  rolling_mean_t30   float64
 16  rolling_mean_t60   float64
 17  rolling_mean_t90   float64
 18  rolling_mean_t180  float64
 19  rolling_std_t7     float64
 20  rolling_std_t30    float64
dtypes: float64(9), int64(12)
memory usage: 979.0 MB


In [ ]:
X_test = test.drop(columns=["dn", "d", "wm_yr_wk", "Unnamed: 0", "Unnamed: 0.1", "id", "day", "weekday", "demand",  "event_name_1", "event_type_1", "event_name_2", "event_type_2"], axis=1)
y_test = test[["demand"]]

In [ ]:
y_train.head(10)

,demand
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,12
8,2
9,0


In [ ]:
TIME_STEP = 7
NUM_FEATRUE = 20

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((X_train.to_numpy(), y_train.to_numpy()))
ds_train = ds_train.window(size=TIME_STEP+1, shift=1, drop_remainder=True)
ds_train = ds_train.flat_map(lambda x, y: tf.data.Dataset.zip((x.batch(TIME_STEP+1), y.batch(TIME_STEP+1))))
ds_train = ds_train.map(lambda x, y: (x[:-1], y[-1]), num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(100000)
ds_train = ds_train.batch(128).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

ds_test = tf.data.Dataset.from_tensor_slices((X_test.to_numpy(), y_test.to_numpy()))
ds_test = ds_test.window(size=TIME_STEP+1, shift=1, drop_remainder=True)
ds_test = ds_test.flat_map(lambda x, y: tf.data.Dataset.zip((x.batch(TIME_STEP+1), y.batch(TIME_STEP+1))))
ds_test = ds_test.map(lambda x, y: (x[:-1], 0), num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
x = []
y = []
for i in tqdm(range(lookback_window, len(df))):
    x.append(df[i - lookback_window:i])
    y.append(df[i])

In [ ]:
print(ds_train)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 21), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))>


In [ ]:
def build_model():
    inputs = tf.keras.Input(shape=(TIME_STEP, NUM_FEATRUE))    
    x = TCN(nb_filters=64,
                 kernel_size=3,
                 nb_stacks=2,
                 dilations=(1, 2, 4, 8, 16, 32, 64),
                 padding='causal',
                 use_skip_connections=True,
                 dropout_rate=0.,
                 return_sequences=False,                 
                 kernel_initializer='he_normal',
                 use_batch_norm=False,
                 use_layer_norm=False,
                 use_weight_norm=False,
                 activation="relu")(inputs)        
    outputs = tf.keras.layers.Dense(4, dtype=tf.float32)(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
model = build_model()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 7, 20)]           0         
                                                                 
 tcn_1 (TCN)                 (None, 64)                338752    
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 339,012
Trainable params: 339,012
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile('adam','mae')

In [ ]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20


ValueError: ignored